In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# IMDB 데이터 셋

### 데이터셋 정보
- 훈련 데이터와 테스트 데이터가 각각 25000개
- 데이터의 각 review에는 label이 붙어 있다.
- 부정은 0, 긍정은 1로 나타낸다.
- 약 50%는 긍정, 50%는 부정 리뷰로 구성
- review 문장의 단어들을 출현빈도순으로 정렬해서 정수로 변환시킨 시퀀스를 x로 한다.
- 스탠포드 대학의 앤드류 마스가 수집했다.

### 예측하고자 하는 방법
- 이진 분류 문제로, 학습 데이터를 이용해, 설계한 신경망을 학습시킨다.
- 학습된 신경망을 이용해 어떤 리뷰가 긍정일 확률을 예측할 수 있도록 한다.

### 학습을 위해 데이터가 어떻게 가공/처리 되었는지?
- 25000개의 샘플을 훈련셋 20000개와 5000개의 검증셋으로 분리
- 길이를 맞추기 위해 패딩 삽입
- 임베딩 층을 통과하여 실수 텐서로 매핑
- LSTM층, Dense층을 차례로 통과하며 모델 훈련

### 사용된 모델의 입력층과 출력층
- 입력층
    - 숫자 인덱스를 밀집 벡터로 매핑하는 embedding층
    - (샘플, 시퀀스길이)를 입력 인자로 받음
    - 크기가 (샘플, 시퀀스 길이, 임베딩 차원)인 3D 실수 텐서를 출력함.
- 출력층
    - 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 출력층
    - 확률(0과 1 사이의 점수, 1에 가까울수록 긍정, 0에 가까울 수록 부정)을 출력하기 위해 시그모이드 활성화 함수(로지스틱 함수, vanishing gradient 문제가 있어서 현재는 잘 사용하지 않음)를 사용

###  사용된 모델의 특징에 대한 기술 
- rmsprop 옵티마이저와 binary_crossentropy 손실 함수로 모델을 컴파일.
- LSTM 층으로 모델을 구성하여 기존의 완전연결 네트워크보다 나은 결과를 기대.
- 다만 하이퍼파라미터를 지정하지 않았으므로(기본값 사용) 매우 큰 차이는 기대하기 힘듬.

In [2]:
from keras.datasets import imdb
import numpy as np

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=20000)

np.load = np_load_old

17465344/17464789 [==============================] - 1s 0us/step


In [0]:
max_features = 20000
text_max_words = 200

#훈련셋, 검증셋 지정
input_val = input_train[max_features:]
y_val = y_train[max_features:]
input_train = input_train[:max_features]
y_train = y_train[:max_features]

In [6]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing import sequence

input_train = sequence.pad_sequences(input_train, maxlen=text_max_words)
input_val = sequence.pad_sequences(input_val, maxlen=text_max_words)
input_test = sequence.pad_sequences(input_test, maxlen=text_max_words)

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=25,
                    batch_size=128,
                    )
model.summary()

W0805 09:34:04.541151 139774824560512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 09:34:04.572379 139774824560512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 09:34:04.578877 139774824560512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 09:34:04.847630 139774824560512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0805 09:34:04.870087 139774824560512 deprecation_wrappe

Epoch 1/25
20000/20000 [==============================] - 28s 1ms/step - loss: 0.4860 - acc: 0.7752
Epoch 2/25
20000/20000 [==============================] - 27s 1ms/step - loss: 0.2735 - acc: 0.8937
Epoch 3/25
20000/20000 [==============================] - 28s 1ms/step - loss: 0.2089 - acc: 0.9221
Epoch 4/25
20000/20000 [==============================] - 27s 1ms/step - loss: 0.1653 - acc: 0.9414
Epoch 5/25
20000/20000 [==============================] - 27s 1ms/step - loss: 0.1368 - acc: 0.9523
Epoch 6/25
20000/20000 [==============================] - 27s 1ms/step - loss: 0.1179 - acc: 0.9602
Epoch 7/25
20000/20000 [==============================] - 27s 1ms/step - loss: 0.0980 - acc: 0.9669
Epoch 8/25
20000/20000 [==============================] - 27s 1ms/step - loss: 0.0875 - acc: 0.9715
Epoch 9/25
20000/20000 [==============================] - 28s 1ms/step - loss: 0.0744 - acc: 0.9751
Epoch 10/25
20000/20000 [==============================] - 28s 1ms/step - loss: 0.0650 - acc: 0.9794